# Eat Safe, Love

## Notebook Set Up

In [ ]:
# Import dependencies
from pymongo import MongoClient
from pprint import pprint

In [ ]:
# Create an instance of MongoClient
mongo = MongoClient(port=27017)

In [ ]:
# assign the uk_food database to a variable name
db = mongo['uk_food']

In [ ]:
# review the collections in our database
print(db.list_collection_names())

In [ ]:
# assign the collection to a variable
establishments = db['establishments']

## Part 3: Exploratory Analysis
Unless otherwise stated, for each question: 
* Use `count_documents` to display the number of documents contained in the result.
* Display the first document in the results using `pprint`.
* Convert the result to a Pandas DataFrame, print the number of rows in the DataFrame, and display the first 10 rows.

### 1. Which establishments have a hygiene score equal to 20?

In [ ]:
# Find the documents in establishments collection with a hygiene score of 20
query_hygiene = {'scores.Hygiene' : 20}

# Use count_documents to display the number of documents in the result
results_hygiene =establishments.count_documents(query_hygiene)
results_hygiene
# Display the first document in the results using pprint
# pprint(establishments.find_one(query_hygiene))


In [ ]:
hygiene_20 = list(establishments.find(query_hygiene))


In [ ]:
# Convert the result to a Pandas DataFrame
hygiene_df = pd.DataFrame(hygiene_20)

# Display the number of rows in the DataFrame
print("Rows in Hygiene DataFrame:", len(hygiene_20))

# Display the first 10 rows of the DataFrame
hygiene_df.head(10)

In [ ]:
hygiene_df.info()

### 2. Which establishments in London have a `RatingValue` greater than or equal to 4?

In [ ]:
# Find the establishments with London as the Local Authority and has a RatingValue greater than or equal to 4.
query_RatingValue = {'RatingValue':{'$gte':4}}

# Use count_documents to display the number of documents in the result
results_RatingValue = establishments.count_documents(query_RatingValue)

# Display the first document in the results using pprint
pprint(establishments.find_one(query_RatingValue))


In [ ]:
RatingValue_4 = list(establishments.find(query_RatingValue))

In [ ]:
# Convert the result to a Pandas DataFrame
RatingValue_df = pd.DataFrame(RatingValue_4)

# Display the number of rows in the DataFrame
print("Rows in RatingValue DataFrame :", len(RatingValue_4))

# Display the first 10 rows of the DataFrame
RatingValue_df.head(5)

In [ ]:
RatingValue_df.info()

### 3. What are the top 5 establishments with a `RatingValue` rating value of 5, sorted by lowest hygiene score, nearest to the new restaurant added, "Penang Flavours"?

In [ ]:
# Search within 0.01 degree on either side of the latitude and longitude.
# Rating value must equal 5
# Sort by hygiene score

degree_search = 0.01
latitude = 51.49014200
longitude = 0.08384000

match_query = {'RatingValue':5,
                         'latitude':{'$gte':latitude-degree_search, '$lte': latitude + degree_search},
                         'longitude':{'$gte':longitude-degree_search, '$lte':longitude + degree_search}}

sort =  [("score.Hygiene",1)]

establishments.find(match_query)

results_RatingValue_5 = list(establishments.find(match_query).sort(sort))
pprint(results_RatingValue_5)


In [ ]:
# Display the number of rows in the DataFrame
print("Number of rows:", len(results_RatingValue_5))

# Display the first 10 rows of the DataFrame
pprint(results_RatingValue_5[0:10])

In [ ]:
# Convert result to Pandas DataFrame
results_RatingValue_5_df = pd.DataFrame(results_RatingValue_5)
results_RatingValue_5_df.head(5)

### 4. How many establishments in each Local Authority area have a hygiene score of 0?

In [ ]:
# Create a pipeline that: 
# 1. Matches establishments with a hygiene score of 0
# 2. Groups the matches by Local Authority
# 3. Sorts the matches from highest to lowest

match_query = {'$match':{'scores.Hygiene':0}}    ##1 

group_query = {'$group':{'_id':'$LocalAuthorityName',  ##2
                         'count':{'$sum':1}}

            }

sort_query = {'$sort':{'count':-1}}    ##3


# Create Pipeline
pipeline = [match_query, group_query, sort_query]

# Print the number of documents in results
result_0 = list(establishments.aggregate(pipeline))

# # Print the first 10 results
pprint(result_0)


In [ ]:
# Display the number of rows in the DataFrame
print("Number of rows:",len(result_0))

# Display the first 10 rows of the DataFrame
pprint(result_0[0:10])


In [ ]:
# Convert the result to a Pandas DataFrame
result_0_df = pd.DataFrame(result_0)